# Converting dlib face recognition resnet model to Tensorflow

This notebook will aim to recreate dlib's face recognition model in Tensorflow. We will directly compare the results between dlib model and Tensorflow model to ensure that they work identically.

Let's first load a bunch of images to test with.

In [ ]:
import glob
import os
from PIL import Image
import numpy as np

faces_folder_paths = [ "George_HW_Bush", "Condoleezza_Rice", "Uma_Thurman" ]
files = list()
for folder in faces_folder_paths:
    files.extend(glob.glob(os.path.join("../data/lfw/", folder, "*.jpg")))
    
def load_image_as_array(filename):
    im = Image.open(filename)
    arr = np.asarray(im)
    arr.flags.writeable = True
    return arr


images = [ load_image_as_array(filename) for filename in files ]

## Load dlib for comparison

In [ ]:
import dlib

predictor_path = "../data/dlib/shape_predictor_68_face_landmarks.dat"
face_rec_model_path = "../data/dlib/dlib_face_recognition_resnet_model_v1.dat"

detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

def dlib_detections_from_image(img):
    detections = detector(img, 1)

    return (img, detections)

def detections_to_descriptors(img, detections):
    return [ facerec.compute_face_descriptor(img, sp(img, d)) for d in detections ]

dlib_detections = [ dlib_detections_from_image(img) for img in images ]
dlib_descriptors = [ detections_to_descriptors(*t) for t in dlib_detections ]

for dtc in dlib_detections:
    img, detections = dtc
    fig, ax = plt.subplots(1)
    
    ax.imshow(img)
    for d in detections:
        width = d.right() - d.left()
        height = d.bottom() - d.top()
        rect = patches.Rectangle((d.left(),d.top()),width,height,linewidth=1,edgecolor='r',facecolor='none')
        ax.add_patch(rect)
    print("Number of faces: {}".format(len(detections)))
    plt.axis('off')
    plt.show()

## Compare results to Tensorflow face descriptor model

First, load the dlib model that is converted to XML.

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('../data/dlib/dlib_face_recognition_resnet_model_v1.xml')
root = tree.getroot()

print(root.tag)

for child in root:
  print(child.tag, child.attrib)
  for c in child:
   print(c.tag, c.attrib)